In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install openpyxl
!pip3 install requests
!pip3 install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/tmp/ipykernel_4164/1222050232.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [77]:
# Fetch the webpage content
url = "https://en.wikipedia.org/wiki/List_of_international_airports_by_country"
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Find the table containing the list of international airports
tables = soup.find_all("table", {"class": "wikitable"})
# country names are in all h4 in div id mw-content-text
# country_names = soup.find_all("div", {"id": "mw-content-text"})[0].find_all("h4")

# Extract the data from the table
data = []
for table in tables:
    country_name = table.find_previous_sibling("h4").find("span", {"class": "mw-headline"}).text
    rows = table.find_all("tr")
    i = 1
    while i < len(rows):
        row = rows[i]
        cols = row.find_all("td")
        if cols[0].get("rowspan"):
            rowspan = int(cols[0].get("rowspan"))
            location = cols[0].text.strip()
            airport = cols[1].text.strip()
            iata_code = cols[2].text.strip()
            data.append([location, country_name, airport, iata_code])

            # Extract the remaining rows for the same location
            for j in range(1, rowspan):
                sub_row = rows[i + j]
                sub_cols = sub_row.find_all("td")
                airport = sub_cols[0].text.strip()
                iata_code = sub_cols[1].text.strip()
                data.append([location, country_name,  airport, iata_code])
            i += rowspan
        else:
            location = cols[0].text.strip()
            airport = cols[1].text.strip()
            iata_code = cols[2].text.strip()
            data.append([location, country_name,  airport, iata_code])
            i += 1

# Create a DataFrame
df = pd.DataFrame(data, columns=["location", "country", "airport", "iata_code"])

In [79]:
df

,location,country,airport,iata_code
0,Algiers,Algeria,Houari Boumediene Airport,ALG
1,Annaba,Algeria,Rabah Bitat Airport,AAE
2,Batna,Algeria,Batna Airport,BLJ
3,Béjaïa,Algeria,Soummam Airport,BJA
4,Chlef,Algeria,Chlef International Airport,CFK
...,...,...,...,...
1485,Funafuti,Sweden,Funafuti International Airport,FUN
1486,Luganville,Sweden,Santo-Pekoa International Airport,SON
1487,Port Vila,Sweden,Bauerfield International Airport,VLI
1488,Futuna,Sweden,Pointe Vele Airport,FUT


In [80]:
# Save to CSV
df.to_csv("international_airports.csv", index=False)

In [81]:
df = pd.read_csv("international_airports.csv")

In [82]:
df

,location,country,airport,iata_code
0,Algiers,Algeria,Houari Boumediene Airport,ALG
1,Annaba,Algeria,Rabah Bitat Airport,AAE
2,Batna,Algeria,Batna Airport,BLJ
3,Béjaïa,Algeria,Soummam Airport,BJA
4,Chlef,Algeria,Chlef International Airport,CFK
...,...,...,...,...
1485,Funafuti,Sweden,Funafuti International Airport,FUN
1486,Luganville,Sweden,Santo-Pekoa International Airport,SON
1487,Port Vila,Sweden,Bauerfield International Airport,VLI
1488,Futuna,Sweden,Pointe Vele Airport,FUT


In [83]:
df[df["location"] == "Paris"]

,location,country,airport,iata_code
1027,Paris,France,Charles de Gaulle Airport,CDG
1028,Paris,France,Orly Airport,ORY


In [100]:
cities_data = pd.read_csv("cities_data.csv")

In [101]:
# convert alternatenames to str type, replace nan with empty string, and create a new column city_names that combines name and alternatenames columns as a single string separated by a comma
cities_data['alternatenames'] = cities_data['alternatenames'].astype(str).replace('nan', '')
cities_data["city_names"] = cities_data["name"] + "," + cities_data["alternatenames"]
cities_data['city_names'] = cities_data['city_names'].str.split(',')

In [102]:
cities_data[cities_data['city_names'].apply(lambda x: 'Port Vila' in x)]

,name,alternatenames,latitude,longitude,country_code,country,beach,family,ski,golf,city_names
28126,Port-Vila,"Bila,Fila,Port Fila,Port Vila,Port-Vila,VLI,Vi...",-17.73648,168.31366,VU,Vanuatu,False,False,False,False,"[Port-Vila, Bila, Fila, Port Fila, Port Vila, ..."


In [103]:
def get_city_data(city_name, country_name, cities_data):
    has_city = cities_data['city_names'].apply(lambda x: city_name in x)
    has_country = cities_data['country'] == country_name
    city_data = cities_data[has_city & has_country]
    return city_data

In [104]:
# Merge the two DataFrames on the location column and the name et alternatenames columns on cities_data DataFrame

# Merge the two DataFrames and get column name, country, country_code from cities_data DataFrame
merged_df = pd.DataFrame()
unfound_airport_city_df = pd.DataFrame()
for i in range(len(df)):
    airport = df.iloc[i]
    location = airport["location"]
    country = airport["country"]
    iata_code = airport["iata_code"]
    airport_name = airport["airport"]

    city_data = get_city_data(location, country, cities_data)
    if len(city_data) > 0:
        city_data = city_data.iloc[0]
        merged_df = pd.concat([merged_df, pd.DataFrame({"location": [city_data["name"]], "airport": [airport_name], "iata_code": [iata_code], "country": [city_data["country"]], "country_code": [city_data["country_code"]]})])
    else:
        unfound_airport_city_df = pd.concat([unfound_airport_city_df, pd.DataFrame({"location": [location], "airport": [airport_name], "iata_code": [iata_code], "country": [country]})])

In [105]:
merged_df

,location,airport,iata_code,country,country_code
0,Algiers,Houari Boumediene Airport,ALG,Algeria,DZ
0,Annaba,Rabah Bitat Airport,AAE,Algeria,DZ
0,Batna City,Batna Airport,BLJ,Algeria,DZ
0,Bejaia,Soummam Airport,BJA,Algeria,DZ
0,Chlef,Chlef International Airport,CFK,Algeria,DZ
...,...,...,...,...,...
0,Stockholm,Stockholm Västerås Airport,VST,Sweden,SE
0,Sundsvall,Sundsvall–Timrå Airport,SDL,Sweden,SE
0,Umea,Umeå Airport,UME,Sweden,SE
0,Vaexjoe,Växjö Airport,VXO,Sweden,SE


In [109]:
unfound_airport_city_df

,location,airport,iata_code,country
0,Oumache,Biskra Airport,BSK,Algeria
0,Marsa Alam,Marsa Alam International Airport,RMF,Egypt
0,Saint Catherine,St. Catherine International Airport,SKV,Egypt
0,Sharm El Sheikh,Sharm El Sheikh International Airport,SSH,Egypt
0,Taba,Taba International Airport,TCP,Egypt
...,...,...,...,...
0,Funafuti,Funafuti International Airport,FUN,Sweden
0,Luganville,Santo-Pekoa International Airport,SON,Sweden
0,Port Vila,Bauerfield International Airport,VLI,Sweden
0,Futuna,Pointe Vele Airport,FUT,Sweden


In [106]:
df.shape[0] - merged_df.shape[0] # Number of airports that were not found in the cities_data DataFrame

286

In [107]:
merged_df[merged_df['country'] == "Japan"]

,location,airport,iata_code,country,country_code
0,Akita,Akita Airport,AXT,Japan,JP
0,Aomori,Aomori Airport,AOJ,Japan,JP
0,Fukuoka,Fukuoka Airport,FUK,Japan,JP
0,Hakodate,Hakodate Airport,HKD,Japan,JP
0,Kagoshima,Kagoshima Airport,KOJ,Japan,JP
0,Komatsu,Komatsu Airport,KMQ,Japan,JP
0,Hiroshima,Hiroshima Airport,HIJ,Japan,JP
0,Kitakyushu,Kitakyushu Airport,KKJ,Japan,JP
0,Nagasaki,Nagasaki Airport,NGS,Japan,JP
0,Naha,Naha Airport,OKA,Japan,JP


In [110]:
# Save to CSV
merged_df.to_csv("international_airports.csv", index=False)
unfound_airport_city_df.to_csv("unfound_airport_city.csv", index=False)

In [ ]:
df = pd.read_csv("international_airports.csv")